In [34]:
import os
import vtk
import numpy as np
from utilities import ReadVTPFile, WriteVTPFile
from vtk.util.numpy_support import vtk_to_numpy, numpy_to_vtk

In [51]:
InputFolder = "/Users/ana/Documents/AnahitaSeresti/05_PrePostCABG/RayCasting/SU03B/PolarMap"
polar_map = os.path.join(InputFolder, "MyocardiumPolarMap.vtp")
PolarMap = ReadVTPFile(polar_map)

In [36]:
def gradient_filter(vtk_image):
    gradient_filter = vtk.vtkImageGradient()
    gradient_filter.SetInputData(vtk_image)
    gradient_filter.SetDimensionality(3)
    gradient_filter.Update()

    return gradient_filter.GetOutput()
    
def define_borders(gradient_image):
    magnitude_filter = vtk.vtkImageMagnitude()
    magnitude_filter.SetInputData(gradient_image)
    magnitude_filter.Update()
    
    return magnitude_filter.GetOutput()

In [40]:
GradientFilter = vtk.vtkGradientFilter()
GradientFilter.SetInputData(PolarMap)
#GradientFilter.SetInputArrayToProcess(0, 0, 0, vtk.vtkDataObject.FIELD_ASSOCIATION_POINTS, "MBFProfile")
GradientFilter.SetInputScalars(vtk.vtkDataObject.FIELD_ASSOCIATION_POINTS, "MBFProfile")
GradientFilter.Update()

GradientFilter.GetOutput()


<vtkmodules.vtkCommonDataModel.vtkPolyData(0x7fcd5c2da2e0) at 0x1a44d4c40>

In [41]:
ofile = os.path.join(InputFolder, "MyocardiumPolarMapGradient.vtp")
WriteVTPFile(ofile, GradientFilter.GetOutput())

In [19]:
MBFPolarValue = vtk_to_numpy(PolarMap.GetPointData().GetArray("MBFProfile"))
MBFPolarValue

array([-9.9999997e+37, -9.9999997e+37, -9.9999997e+37, ...,
        7.9290184e+01,  7.8991417e+01,  7.8770378e+01], dtype=float32)

In [25]:
MBFPolarValue[(MBFPolarValue > 1000) | (MBFPolarValue < -100)] = 0

MBFPolarValueVTK = numpy_to_vtk(MBFPolarValue)
MBFPolarValueVTK.SetName("MBFProfile2")
PolarMap.GetPointData().AddArray(MBFPolarValueVTK)


GradientFilter = vtk.vtkGradientFilter()
GradientFilter.SetInputData(PolarMap)
#GradientFilter.SetInputArrayToProcess(0, 0, 0, vtk.vtkDataObject.FIELD_ASSOCIATION_POINTS, "MBFProfile")
GradientFilter.SetInputScalars(vtk.vtkDataObject.FIELD_ASSOCIATION_POINTS, "MBFProfile2")
GradientFilter.Update()

GradientFilter.GetOutput()
ofile = os.path.join(InputFolder, "MyocardiumPolarMapGradient.vtp")
WriteVTPFile(ofile, GradientFilter.GetOutput())

In [52]:
from sklearn.cluster import KMeans

MBFPolarValue = vtk_to_numpy(PolarMap.GetPointData().GetArray("MBFProfile"))
#MBFPolarValue = MBFPolarValue[(MBFPolarValue < 1000) & (MBFPolarValue > -100)]
X = MBFPolarValue.reshape(-1,1)
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X)
labels = kmeans.labels_

/usr/local/anaconda3/envs/vtkfenics/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [44]:
len(labels[labels==1])

410634

In [53]:
means = [MBFPolarValue[labels == i].mean() for i in range(2)]
means

[156.3745, 121.26375]

In [54]:
ischemic_label = np.argmin(means)
ischemic_label

1

In [55]:
ischemia_mask = labels == ischemic_label
ischemia_mask

array([False, False, False, ...,  True, False,  True])

In [56]:
ischemia_array = numpy_to_vtk(ischemia_mask.astype(np.uint8), deep=True)
ischemia_array.SetName("IsIschemic")
PolarMap.GetPointData().AddArray(ischemia_array)

ofile = os.path.join(InputFolder, "MyocardiumPolarMapIschemic.vtp")
WriteVTPFile(ofile, PolarMap)